# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Вам нужно научиться определять рыночную стоимость объектов недвижимости. Для этого проведите исследовательский анализ данных и установите параметры, влияющие на цену объектов. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта и других объектов — эти данные автоматически получены из геосервисов. Количество парков и водоёмов также заполняется без участия пользователя.

__Цель исследования__

1. Определить рыночную стоимость объектов недвижимости,н
2. Построить автоматизированную систему. которая отследит аномалии и мошенническую деятельность.
3. Установить, какие факторы влияют на ценообразование стоимости квартир.

__Ход исследования__

Данные о недвижимости я получу из файла "real_estate_data.csv". О качестве данных ничего не известно. Поэтому перед тем, как приступать к целям исследования, понадобится обзор данных.

Я проверю данные на ошибки и оценю их влияние на исследование. Затем, на этапе предобработки я поищу возможность исправить все ошибки данных, которые не приведут к искажению конечного результата. Далее, создам необходимые столбцы, и приступлю к проведению исследовательского анализа

Таким образом, моё исследование пройдет в семь этапов:

- Обзор данных.
- Предобработка данных.
- Расчеты и добавление результатов в таблицу.
- Проведение анализа для Ленинградской области,
- Проведение анализа для Санкт-Петербурга.
- Различаются ли факторы, которые влияют на ценообразование стоимости квартир.
- Написание общего вывода.

### Откройте файл с данными и изучите общую информацию

Импорт требуемых библиотек

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Установка стандартных значений размера для выводимых графиков

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

Чтение DataFrame

In [ ]:
df = pd.read_csv('C:\Users\User\Downloads\real_estate_data.csv', sep='\t')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-5-fa9a03ef313b>, line 1)

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.hist(figsize=(20, 20))
pass #Чтобы не было лишних слов в выводе

**Вывод по разделу 1:**

Ознакомились с DataFrame. Изначально можно уже выделить определенное количество проблем, с которыми придется столкнуться при предобработке, но данные выглядят многообещающе.Гистограммы по большей части представляют нормальное распределение(или Bell-curve) там, где оно должно быть.

### Выполните предобработку данных

In [ ]:
df.isna().sum()

In [ ]:
pd.DataFrame(round(df.isna().mean()*100)).sort_values(by=0, ascending=False).style.background_gradient('coolwarm')

**Для простоты пойдем по списку**

- **1** Ceiling_height заменим значения на средние, опираясь на местоположение, так как там обычно свои стандарты применяются при строительстве

In [ ]:
for d in df['locality_name'].unique():
    df.loc[(df['locality_name'] == d) & (df['ceiling_height'].isna()), 'ceiling_height']\
    = round(df[df['locality_name'] == d]['ceiling_height'].mean(), 2)

- **2** Floors_total оставим без изменений, так как 86 пропусков не так уж и много

- **3** Living_area также оставим как есть. Решение проблемы пропусков займет слишком много времени, а еще не очень понятно, на каком основании вставлять значения.

- 4 Is_apartment  проверим, есть ли заивсимость по цене у того, что имеем

In [ ]:
df['is_apartment'].value_counts()

In [ ]:
df[df['is_apartment'] == True].sort_values('last_price') #Проверим цены у апартаментов, можем ли по ним базировать выбор

In [ ]:
df['is_apartment'] = df['is_apartment'].fillna(False)

Нет явной зависимости = ставим False

- 5 Kitchen_area оставим как есть. Решение проблемы пропусков займет слишком много времени, а еще не очень понятно, на каком основании вставлять значения.

- 6 Balcony посмотрим, что имеем:

In [ ]:
df['balcony'].value_counts()

In [ ]:
df.plot(y='balcony', x='rooms', kind='scatter')
df.plot(x='balcony', y='last_price', kind='scatter')
pass

Мы не можем фактически поставить зависимость между количеством балконов и ценой/количеством комнат, поэтому заменим все пропуски на 0.(И да, 5 балконов действительно может существовать)

In [ ]:
df['balcony'] = df['balcony'].fillna(0)
df['balcony'].value_counts()

- 7 Locality_name  оставим, как есть, но пропуски заменим на unknown

In [ ]:
df['locality_name'] = df['locality_name'].fillna('Unknown')

- 8 Airports_nearest нужно оставить NaN, так как мы не можем заменить на 0. NaN появился из-за того что нет аэропорта в ближайшей доступности, но это не значит что расстояние 0. Также не значит, что можем заменить на медианное. Можем только оставить NaN

- 9 cityCenters_nearest проверим одну теорию:

In [ ]:
df[(df['cityCenters_nearest'].isna()) & (df['parks_around3000'].isna()) & (df['ponds_around3000'].isna())]

Как мы видим - все пропуски взаимосвязаны в этих 3 категориях. Можем лишь заменить два столбца на 0, так как они категориальные и по факту будут отображать количество ближайших парков/озер в 0. Остальные оставим так, как есть из-за той же причины, что и с аэропортами.

In [ ]:
df[['parks_around3000','ponds_around3000']] = df[['parks_around3000','ponds_around3000']].fillna(0)

- 10 Days_exposition самое логичное - что объявление еще висит. Значит оставляем NaN, так как нам нужно будет проделывать операции

Дальше разберемся с типами данных:

In [ ]:
df['last_price'] = df['last_price'].astype(int)

In [ ]:
df['balcony'] = df['balcony'].astype(int)

In [ ]:
df['parks_around3000'] = df['parks_around3000'].astype(int)
df['ponds_around3000'] = df['ponds_around3000'].astype(int)

Заменили все численные на реально численные, кроме тех колонок, где мы оставили NaN, так как там не получится заменить.

In [ ]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y%m%dT%H:%M:%S')

Преобразуем дату в формат даты

In [ ]:
df['locality_name'] = df['locality_name'].str.replace('ё', 'е')\
.replace(['поселок городского', 'городской поселок', 'поселок городского типа'], 'поселок', regex=True)

Заменим названия на приведенные к общему стандарту

Посмотрим на очевидные выбросы:

In [ ]:
df.boxplot(column='last_price', figsize=(10,10))
pass

Тут видим, что есть совсем чуть-чуть значений, которые сильно выбиваются. Лучше от них избавиться

In [ ]:
df.sort_values('last_price', ascending=False).head(15)

In [ ]:
df = df[df['last_price'] < 160000000]

In [ ]:
df.boxplot(column='last_price', figsize=(10,10))
pass

Также посмотрим на минимальный порог. Есть ли ненормальные цены:

In [ ]:
df[df['last_price'] < 400000]

In [ ]:
df = df[(df['last_price'] < 100000000) & (df['last_price'] > 400000)]

Выбрали цифру 100000000, так как на графике видно, что выше лишь аномалии

Пройдемся по другим столбцам, чтобы определить явные выбросы

In [ ]:
df_to_analyze = df.drop(columns='last_price')#Создадим заглушку для того, чтобы проверить графики размаха других столбцов
df_to_analyze.boxplot(figsize=(20,10))
pass

In [ ]:
df_to_analyze_2 = df_to_analyze.drop(columns=['airports_nearest', 'cityCenters_nearest'])#Заглушка №2
df_to_analyze_2.boxplot(figsize=(20,10))
pass

Явных аномалий нет.

In [ ]:
df = df.rename(columns={'locality_name': 'location','cityCenters_nearest': 'city_center_nearest', \
                        'parks_around3000':'parks_around_3km', 'ponds_around3000':'ponds_around_3km' })

Напоследок переименуем столбцы для удобства и чтобы было по правилам

Также стоит проверить DataFrame на явные дубликаты. Хоть на первый взгляд их нет, но нужно быть уверенным в этом.

In [ ]:
df.duplicated().sum()

И все же задвоений не оказалось. Отлично, меньше работы.

**Вывод по разделу 2:**

В результате проведенной предобработки разобрались с бОльшим количеством пропусков. Избавились от явных аномалий, явных дубликатов, пытаясь сохранить как можно больше строк. Переименовали значения для одного стиля, а также колонки для удобства работы. DataFrame готов к анализу.

### Добавьте в таблицу новые столбцы

#### Добавляем столбец со стоимостью квадратного метра

In [ ]:
df['quad_m_cost'] = round(df['last_price'] / df['total_area'], 2)

#### Добавляем столбцы с днем, месяцем и годом создания объявления

In [ ]:
df['publication_day'] = df['first_day_exposition'].dt.day_name()

In [ ]:
df['publication_month'] = df['first_day_exposition'].dt.month_name()

In [ ]:
df['publication_year'] = df['first_day_exposition'].dt.to_period("Y")
df['publication_year'] = df['publication_year'].astype(str)#Потом оказывается, что только тип данных года не str

Мы все знаем английский язык(надеюсь), так что оставим так. Конечно, можно переименовать на русский мотив, но это лишняя работа

In [ ]:
df.head(10)

#### Добавляем столбец с типом этажа

In [ ]:
def floor(row):
    if row['floor'] == row['floors_total']:
        return 'Last'
    elif row['floor'] == 1:
        return 'First'
    else:
        return 'Else'

In [ ]:
df['floor_type'] = df.apply(floor, axis=1)

In [ ]:
df['floor_type'].unique()

Все значения как надо, значит ошибок не вышло

#### Добавляем столбец с расстоянием до центра в км

In [ ]:
#df['city_center_nearest_km'] = (df['city_center_nearest'] / 1000).astype('int64', errors='ignore')

In [ ]:
df['city_center_nearest_km'] = round((df['city_center_nearest'] / 1000), 0)

**Вывод по разделу 3:**

Отредактировали DataFrame, добавив нужные столбцы для анализа. Кое-где категоризировали переменные, где-то создали соверешнно новые значения в результате взаимодействий со столбцами. Подготовили почву для проведения исследовательского анализа.

### Проведите исследовательский анализ данных

#### Изучение параметров DataFrame ####
- **1** Total_area

In [ ]:
pd.Series(df['total_area']).plot(kind='hist', bins=50, range=(0,350), grid=True, title='Общая площадь')
plt.xlabel('Метры')
plt.ylabel('Частота появления')
pass

In [ ]:
df.boxplot(column='total_area', figsize=(10,15))
pass

В целом ничего необычного, есть небольшое количество выбросов, но их можно оставить.

- **2** Living_area

In [ ]:
pd.Series(df['living_area']).plot(kind='hist', bins=50, range=(0,200), grid=True, title='Площадь жилой части')
plt.xlabel('Метры')
plt.ylabel('Частота появления')
pass

In [ ]:
df[df['living_area'] < 25]['location'].value_counts()

Здесь, в целом, тоже все в порядке. Скачки из-за типов квартир. Код сверху это доказывает(так как типовые квартиры расположены в городах и там одинаковый метраж и планировка).

- **3** Kitchen_area

In [ ]:
pd.Series(df['kitchen_area']).plot(kind='hist', bins=50, range=(0,55), grid=True, title='Площадь кухни')
plt.xlabel('Метры')
plt.ylabel('Частота появления')
pass

Здесь тоже все в порядке. Нормальное распределение.

- **4** Last_price

In [ ]:
pd.Series(df['last_price']).plot(kind='hist', bins=100, grid=True, title='последняя цена')
plt.xlabel('Цена')
plt.ylabel('Частота появления')
pass

In [ ]:
df[df['last_price'] > 40000000]

Видим, что все равно 28 значений немного выбивают график. Мы уже избавились от совсем невозможных значений ранее. Лучше их оставить.

- **5** Rooms

In [ ]:
pd.Series(df['rooms']).plot(kind='hist', bins=50, grid=True, title='комнаты')
plt.xlabel('Количество комнат')
plt.ylabel('Частота появления')
pass

Увидели, что есть квартиры с 0 комнат. В теории это возможно(изба это 0-комнатное жилое помещение). Но для этого надо кое-что проверить:

In [ ]:
df[df['rooms'] == 0]['location'].value_counts()

В Питере это буквально абсурд. В Санкт-Петербурге такое невозможно

In [ ]:
df[df['rooms'] == 0] = df[(df['rooms'] == 0) & ~(df['location'] == 'Санкт-Петербург')]
df = df.dropna(subset=['total_images']) #Так как в этом столбце не было NaN до этого, а в rooms еще остались

In [ ]:
df=df[df['rooms'] <7]

Бессмысслено измерять с количеством комнат больше 7

- **6** Ceiling_height

In [ ]:
pd.Series(df['ceiling_height']).value_counts().reset_index().sort_values(by='index')

В теории значения действительно могли записать как вещественные, так что поступим так - разделим все высокие значения на 10, а потом удалим все строки, не соответствующие стандарту - больше 4.5 метров

In [ ]:
df.loc[df['ceiling_height'] > 10, 'ceiling_height'] = df[df['ceiling_height'] > 10]['ceiling_height'] / 10

In [ ]:
df = df[df['ceiling_height'] < 4.5]

In [ ]:
pd.Series(df['ceiling_height']).plot(kind='hist', bins=20, grid=True, title='Высота потолков', range=(0, 4.5))
plt.xlabel('Метры')
plt.ylabel('Частота появления')
pass

- **7** Floor_type

In [ ]:
counts = df['floor_type'].value_counts()
plt.bar(counts.index, counts.values)
plt.ylabel('Частота появления')
plt.xlabel('Тип этажа')
plt.title('Типы этажей')
pass

Очевидно, что количество квартир с последним или первым этажом куда меньше, чем остальных. Кстати, количество первых примерно равно количеству последних, значит все в порядке

- **8** Floors_total

In [ ]:
pd.Series(df['floors_total']).plot(kind='hist', bins=100, grid=True, title='Общее количество этажей')
plt.xlabel('Этажей в здании')
plt.ylabel('Частота появления')
pass

In [ ]:
df[df['floors_total'] >30]

Исходя из того, что средний уровень этажа равен 3 метра и взяв междуэтажное расстояние примерно в 1 метр получаем, что для здания с 30+ этажей высота должна составлять 120+ метров, что уже является небоскребом(нежилым зданием). Отбросим эти значения, так как они не подходят для анализа

In [ ]:
df = df[df['floors_total'] < 30]

- **9** City_center_nearest

In [ ]:
pd.Series(df['city_center_nearest']).plot(kind='hist', bins=100, grid=True, title='Ближайший центр города')
plt.xlabel('Метры')
plt.ylabel('Частота появления')
pass

Тяжело что-либо сказать. Речи о распределении быть не может на этом графике.

In [ ]:
df.boxplot(column='city_center_nearest',figsize=(10,10))
pass

Оставляем как есть

- **10** Parks_nearest

In [ ]:
pd.Series(df['parks_nearest']).plot(kind='hist', bins=100, grid=True, title='Ближайший парк')
plt.xlabel('Метры')
plt.ylabel('Частота появления')
pass

Та же ситуация, что и с центрами города, хотя тут больше похоже на нормальное распределение.

#### Изучим, как быстро продавались квартиры

In [ ]:
pd.Series(df['days_exposition']).plot(kind='hist', bins=100, grid=True, title='Время объявления')
plt.xlabel('Дни')
plt.ylabel('Частота появления')
pass

In [ ]:
selling_speed = df.groupby('location').days_exposition.agg(mean=(lambda x: round(x.mean(), 2)), median='median')

In [ ]:
selling_speed = selling_speed.dropna()
selling_speed.sort_values('mean', ascending=False)

In [ ]:
selling_speed.sort_values('median', ascending=False)

In [ ]:
pd.Series(selling_speed['mean']).plot(kind='hist', bins=100, grid=True, title='Среднее продаж по населенным пунктам')
plt.ylabel('Частота появления')
plt.xlabel('Цена')
pass

In [ ]:
pd.Series(selling_speed['median']).plot(kind='hist', bins=100, grid=True, title='Медиана продаж по населенным пунктам')
plt.ylabel('Частота появления')
plt.xlabel('Цена')
pass

Рассмотрим более детально медиану:

In [ ]:
selling_speed.boxplot(column='median')
pass

In [ ]:
selling_speed['median'].describe()

Изходя из более подробного рассмотрения, можно увидеть, что медиана медиан продаж по городам находится на отметке 162. В целом логично предположить, что продажи до отметки q1 = 76 считаются "быстрыми", то есть быстрее, чем обычно. Продажи после этой отметки, но не позднее значения q3 = 185 - "Средние". Продажи позднее средних, но до значения полутораквартильного расстояния (q3-q1)\*1,5+q3 = 348 - "Долгие". Продажи после этой отметки - "Необычайно долгие". Резюмируем информацию:
* Продажи <= 76 дней = "Быстрые"
* 76 дней < Продажи <= 185 дней = "Средние"
* 185 дней < Продажи <= 348 дней = "Долгие"
* Продажи > 348 дней = "Необычайно долгие"

#### Изучим факторы, влияющии на цену ####

Для подготовки с некоторыми сравнениям сделаем сводные таблицы. но для некоторых будем использовать диаграмму рассеяния, так как значений очень много

- **1** Связь с общей площадью

In [ ]:
df.plot(x='last_price', y='total_area', kind='scatter', grid=True, title='Общая площадь/цена')
plt.ylabel('Общая площадь')
plt.xlabel('Последняя цена')
pass

In [ ]:
df[['last_price', 'total_area']].corr()

Ну тут у нас почти прямая зависимость в начале, дальше больше рассеивается, но в целом зависимость продолжается. Коэффициент корреляции поддерживает данный вывод.

- **2** Связь с жилой площадью

In [ ]:
df.plot(x='last_price', y='living_area', kind='scatter', grid=True, title='Жилая площадь/Цена')
plt.ylabel('Жилая площадь')
plt.xlabel('Последняя цена')
pass

In [ ]:
df[['last_price', 'living_area']].corr()

Есть небольшие выбросы, но в целом зависимость также прямая, хотя коэффициент уже падает.

- **3** Связь с площадью кухни

In [ ]:
df.plot(x='last_price', y='kitchen_area', kind='scatter', grid=True, title='Площадь кухни/Цена')
plt.ylabel('Площадь кухни')
plt.xlabel('Последняя цена')
pass

In [ ]:
df[['last_price', 'kitchen_area']].corr()

Вот тут уже не очень прямая. В целом не сильно зависит цена от площади кухни. Коэффициент также почти 0.5 - полный хаос.

- **4** Связь с количеством комнат

In [ ]:
rooms = df.pivot_table(values='last_price', index='rooms', aggfunc='median')
plt.bar(rooms.index, rooms['last_price'].values)
plt.ylabel('Цена')
plt.xlabel('Количество комнат')
plt.title('Комнаты/Цена')
pass

Ну вот тут, как и хотелось, логичная прямая зависимость

- **5** Связь с типом этажа

In [ ]:
floors = df.pivot_table(values='last_price', index='floor_type', aggfunc='median')
plt.bar(floors.index, floors['last_price'].values)
plt.ylabel('Цена')
plt.xlabel('Тип этажа')
plt.title('Тип этажа/Цена')
pass

На удивление одинаково, что странно, потому что квартир, очевидно, меньше. Это говорит о том, что люди готовы переплатить для квартир, расположенных на первом и последнем этажах

- **6** Связь с днем размещения объявления

In [ ]:
days = df.pivot_table(values='last_price', index='publication_day', aggfunc='median')
plt.bar(days.index, days['last_price'].values)
plt.ylabel('Цена')
plt.xlabel('День недели')
plt.title('Продажи по дням')
pass

Не зависит от дня недели

- **7** Связь с месяцем размещения

In [ ]:
months = df.pivot_table(values='last_price', index='publication_month', aggfunc='median')
plt.bar(months.index, months['last_price'].values)
plt.ylabel('Цена')
plt.xlabel('Месяц')
plt.title('Продажи по месяцам')
pass

Не зависит от месяца

- **8** Связь с годом размещения

In [ ]:
years = df.pivot_table(values='last_price', index='publication_year', aggfunc='median')
plt.bar(years.index, years['last_price'].values)
plt.ylabel('Цена')
plt.xlabel('Год')
plt.title('Продажи по годам')
pass

А вот по годам есть зависимость. Куда больше дома стоили раньше, в самом первом году - 2014. Посмотрим глубже.

In [ ]:
years_square = df.pivot_table(values='total_area', index='publication_year', aggfunc='median')
plt.bar(years_square.index, years_square['total_area'].values)
plt.ylabel('Общая площадь')
plt.xlabel('Год')
plt.title('Площадь продаваемых квартир по годам')
pass

Оказалось, что цена была напрямую связана с площадью, так что год все же не влияет на стоимость.

#### Посчитаем среднюю цену квадратного метра

Выведем сводную таблицу, в которой будет 10 наибольших количеств объявлений по городам

In [ ]:
quad_price = df.groupby('location').agg(number=('rooms','count'), avg_quad_m_price=('quad_m_cost', 'median')).\
sort_values('number', ascending=False).head(10)
quad_price

Город с наибольшей ценой квадартного метра

In [ ]:
quad_price[quad_price['avg_quad_m_price'] == quad_price['avg_quad_m_price'].max()]

С наименьшей ценой квадратного метра

In [ ]:
quad_price[quad_price['avg_quad_m_price'] == quad_price['avg_quad_m_price'].min()]

#### Вычислим среднюю стоимость квартир в Санкт-Петербурге по удаленности от центра

In [ ]:
price = df[df['location'] == 'Санкт-Петербург'].groupby('city_center_nearest_km').\
agg(price_for_km=('last_price', lambda x: int(x.median())), number=('rooms', 'count'))

In [ ]:
price = price[price['number'] > 30] #Отбросим те строки, где количество квартир небольшое(Да, мы отбросили 0 км тоже)
plt.bar(price.index, price['price_for_km'].values)
plt.title('Распределение цены по километрам от центра')
plt.xlabel('Километры от центра')
plt.ylabel('Цена')
pass

Из данной гистограммы можно сделать очевидный вывод о том, что чем ближе квартира к центру, тем она дороже. Но также возникает интересный момент на км = 3. Видимо, либо на такой удаленности мешает ландшафт(много рек), либо люди готовы переплатить только за 1-2 км.

**Вывод по разделу 4:**

В ходе проведения исследовательского анализа рассмотрели самые важные компоненты DataFrame, а также сравнили влияние разных параметров на конечную цену(Результаты будут опубликованы в финальном выводе). Выделил города/поселки с максимальной/минимальной ценой за квадратный метр. Рассмотрели конкретно Санкт-Петербург, так как изначально было интересно, влияет ли расстояние от центра на стоимость. Получились необычные результаты.

### Напишите общий вывод

- **1. Знакомство с DataFrame:**
- **1.1** Рассмотрели данные со стороны применимости для анализа. Несмотря на определенное количество пропусков, полученная информация оказалась достаточной для начала анализирования.
- **1.2** Построили гистограммы по каждому столбцу, чтобы понять, насколько данные приближены к реальности. В тех, где нужно, нашли около нормальное распределение, соответственно все в порядке.
- **2. Предобработка DataFrame:**
- **2.1** Во всех столбцах избавились от пропусков, пытаясь сохранить как можно больше строк. Удалили аномальные значения или заменили их на подходящие под соответствующие столбцы.
- **2.2** Избавились от явных выбросов, явных дубликатов, которые могли бы повлиять на результаты исследований.
- **2.3** Переименовали значения и столбцы для соответствия стандартам
- **3. Добавление столбцов:**
- **3.1** Добавили столбцы с категориями для упрощения вычислений.
- **3.2** Провели базовые манипуляции для добавления столбцов, наиболее интересующих нас.
- **3.3** Разделили дату на дни, месяцы и годы, создав соответствующие столбцы.
- **4. Исследовательский анализ:**
- **4.1** Изучение параметров:
- **4.1.1** Рассмотрели все значимые параметры в DataFrame со стороны статистики и откорректировали значения для дальнейшего анализа.
- **4.1.2** В результате получили полноценный DataFrame, в котором все значения находятся в пределах разумных.
- **4.2** Рассмотрели, насколько быстро продавались квартиры. Построили графики, оценили скорость.
- **4.2.1** Вывод из раздела:
    * Продажи <= 76 дней = "Быстрые"
    * 76 дней < Продажи <= 185 дней = "Средние"
    * 185 дней < Продажи <= 348 дней = "Долгие"
    * Продажи > 348 дней = "Необычайно долгие"
- **4.3** Рассмотрели все значимые факторы, которые могут повлиять на общую цену.
- **4.3.1** Вывод из раздела - на цену влияют:
    * Общая площадь
    * Жилая площадь
    * Количество комнат
    
Остальные факторы оказались недостаточно значимыми
- **4.4** Оценка среднего квадратного метра по населенным пунктам.
- **4.4.1** Изучили среднюю квадратуру регионов с наибольшим количеством объявлений, а также узнали, что в Санкт-Петербурге самый дорогой квадратный метр, а в Выборге - самый дешеый.
- **4.5** Оценили стоимость объектов в Санкт-Петербурге в завимиости от удаленности от центра.
- **4.5.1** Построили графики и нашли зависимость между ценой объекта и удаленностью от центра. Зависимость оказалась не до конца прямой. Удаленность в 3-5 километров выдает необычный результат, что может говорить о том, что большинство людей готовы переплачивать только за расстояние в 1-2 км.
- **5.1 Результаты проведенного анализа:**
    * Разобрались, с какой скоростью должна продаваться квартира, используя статистику и графики размаха
    * Установили важнейшие факторы, влияющие на стоимость
    * Рассчитали среднюю квадратуру для каждого населенного пункта
    * Нашли необычную зависимость между расстоянием от центра и стоимостью

**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Файл с данными открыт.
- [x]  Файл с данными изучен: выведены первые строки, использован метод `info()`, построены гистограммы.
- [x]  Найдены пропущенные значения.
- [x]  Пропущенные значения заполнены там, где это возможно.
- [x]  Объяснено, какие пропущенные значения обнаружены.
- [x]  В каждом столбце установлен корректный тип данных.
- [x]  Объяснено, в каких столбцах изменён тип данных и почему.
- [x]  Устранены неявные дубликаты в названиях населённых пунктов.
- [x]  Обработаны редкие и выбивающиеся значения (аномалии).
- [x]  В таблицу добавлены новые параметры:
       – цена одного квадратного метра;
       – день публикации объявления (0 - понедельник, 1 - вторник и т. д.);
       – месяц публикации объявления;
       – год публикации объявления;
       – тип этажа квартиры (значения — «первый», «последний», «другой»);
       – расстояние до центра города в километрах.
- [x]  Изучены и описаны параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего парка.
- [x]  Выполнено задание «Изучите, как быстро продавались квартиры (столбец `days_exposition`)»:
    - построена гистограмма;
    - рассчитаны среднее и медиана;
    - описано, сколько обычно занимает продажа и указано, какие продажи можно считать быстрыми, а какие — необычно долгими.
- [x]  Выполнено задание «Определите факторы, которые больше всего влияют на общую (полную) стоимость объекта». Построены графики, которые показывают зависимость цены от параметров:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - количество комнат;
        - тип этажа, на котором расположена квартира (первый, последний, другой);
        - дата размещения (день недели, месяц, год).
- [x]  Выполнено задание «Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений»:
    - выделены населённые пункты с самой высокой и низкой стоимостью квадратного метра.
- [x]  Выполнено задание «Выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите их среднюю стоимость на разном удалении от центра»:
    -  учтён каждый километр расстояния, известны средние цены квартир в одном километре от центра, в двух и так далее;
    -  описано, как стоимость объекта зависит от расстояния до центра города;
    -  построен график изменения средней цены для каждого километра от центра Петербурга.
- [x]  На каждом этапе сделаны промежуточные выводы.
- [x]  В конце проекта сделан общий вывод.